In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp
import pandas as pd

In [2]:
# figure out the correct path
machop_path = Path(".").resolve().parent.parent /"rs1923/mase_real/machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity, get_logger

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model

from chop.tools.checkpoint_load import load_model

set_logging_verbosity("debug")

logger = get_logger("chop")
logger.setLevel(logging.INFO)

batch_size = 512
model_name = "jsc-tiny"
dataset_name = "jsc"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

dummy_in = {"x": next(iter(data_module.train_dataloader()))[0]}

/home/ruiqi/anaconda3/envs/mase/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total number of JSC_1923 parameters: 3285
Total number of JSC_Tiny parameters: 117


INFO     Set logging level to debug


In [5]:
from torch import nn
from chop.passes.graph.utils import get_parent_name

# 1. Can you edit your code, so that we can modify the above network to have layers expanded to double their sizes? Note: you will have to change the ReLU also.
class JSC_Three_Linear_Layers(nn.Module):
    def __init__(self):
        super(JSC_Three_Linear_Layers, self).__init__()
        self.seq_blocks = nn.Sequential(
            nn.BatchNorm1d(16),  # unchanged      
            nn.ReLU(16),     
            nn.Linear(16, 16), # output      
            nn.ReLU(16),     
            nn.Linear(16, 16), # input & output     
            nn.ReLU(16),        
            nn.Linear(16, 5),  # input    
            nn.ReLU(5),       
        )

    def forward(self, x):
        return self.seq_blocks(x)
    

In [3]:
# prepare for the network transform (i.e, channel multiplier)

import copy

def instantiate_linear(in_features, out_features, bias):
    if bias is not None:
        bias = True
    return nn.Linear(
        in_features=in_features, 
        out_features=out_features, 
        bias=bias)

def redefine_linear_transform_pass(graph, pass_args=None):
    pass_args_copy = copy.deepcopy(pass_args)
    main_config = pass_args_copy.pop('config')
    print("main_config",main_config)
    
    default = main_config.pop('default', None)
    if default is None:
        raise ValueError(f"default value must be provided.")
    i = 0
    for node in graph.fx_graph.nodes:
        i += 1
        # if node name is not matched, it won't be tracked
        config = main_config.get(node.name, default)['config']
        name = config.get("name", None)
        if name is not None:
            ori_module = graph.modules[node.target]
            in_features = ori_module.in_features
            out_features = ori_module.out_features
            bias = ori_module.bias
            if name == "output_only":
                out_features = out_features * config["channel_multiplier"] 
            elif name == "both":
                in_features = in_features * config["channel_multiplier"] 
                out_features = out_features * config["channel_multiplier"] 
            elif name == "input_only":
                in_features = in_features * config["channel_multiplier"] 
                # bias unchanged
            new_module = instantiate_linear(in_features, out_features, bias)
            parent_name, name = get_parent_name(node.target)
            setattr(graph.modules[parent_name], name, new_module)
    return graph, {}


def instantiate_relu(boolean):
    return nn.ReLU(inplace=boolean)

def redefine_relu_pass(graph, pass_args=None):
    pass_args_copy = copy.deepcopy(pass_args)
    main_config = pass_args_copy.pop('config')
    default = main_config.pop('default', None)
    if default is None:
        raise ValueError(f"default value must be provided.")
    i = 0
    for node in graph.fx_graph.nodes:
        i += 1
        # if node name is not matched, it won't be tracked
        config = main_config.get(node.name, default)['config']
        name = config.get("name", None)
        if name is not None:
            new_module = instantiate_relu(True)
            parent_name, name = get_parent_name(node.target)
            setattr(graph.modules[parent_name], name, new_module)
    return graph, {}
    

pass_config_linear = {
"by": "name",
"default": {"config": {"name": None}},
"seq_blocks_2": {
    "config": {
        "name": "output_only",
        "channel_multiplier": 2,
        }
    },
"seq_blocks_4": {
    "config": {
        "name": "both",
        "channel_multiplier": 2,
        }
    },
"seq_blocks_6": {
    "config": {
        "name": "input_only",
        "channel_multiplier": 2,
        }
    },
}

pass_config_relu = {
"by": "name",
"default": {"config": {"name": None}},
"seq_blocks_3": {
    "config": {
        "name": "relu",
        }
    },
"seq_blocks_5": {
    "config": {
        "name": "relu",
        }
    },
}

In [6]:
model = JSC_Three_Linear_Layers()
mg = MaseGraph(model=model)
mg, _ = init_metadata_analysis_pass(mg, None)

mg, _ = redefine_linear_transform_pass(graph=mg, pass_args={"config": pass_config_linear})
mg, _ = redefine_relu_pass(graph=mg, pass_args={"config": pass_config_relu})

main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier': 2}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier': 2}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier': 2}}}


In [7]:
mg.model

GraphModule(
  (seq_blocks): Module(
    (0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=16, out_features=32, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=32, out_features=5, bias=True)
    (7): ReLU(inplace=True)
  )
)

In [8]:
# 2. Grid Search for the best channel multiplier value

# train

'''
import torch
from torchmetrics.classification import MulticlassAccuracy
from torch import optim

def init_mg():
    model = JSC_Three_Linear_Layers()
    mg = MaseGraph(model=model)
    mg, _ = init_metadata_analysis_pass(mg, None)
    return mg

channel_multiplier = [1,2,3,4,5]
max_epoch=10

batch_size = 512
optimizer = optim.Adam(model.parameters(), lr=0.001)

for multiplier in channel_multiplier:
    mg = init_mg()
    pass_config_linear["seq_blocks_2"]["config"]["channel_multiplier"] = multiplier
    pass_config_linear["seq_blocks_4"]["config"]["channel_multiplier"] = multiplier
    pass_config_linear["seq_blocks_6"]["config"]["channel_multiplier"] = multiplier

    mg, _ = redefine_linear_transform_pass(graph=mg, pass_args={"config": pass_config_linear})
    mg, _ = redefine_relu_pass(graph=mg, pass_args={"config": pass_config_relu})

    for epoch in range(max_epoch):
        for inputs in data_module.train_dataloader():
            xs, ys = inputs
            optimizer.zero_grad()
            preds = mg.model(xs)
            loss = torch.nn.functional.cross_entropy(preds, ys)  
            loss.backward()  
            optimizer.step() 

    torch.save({
        'state_dict': mg.model.state_dict(),
        'config': multiplier,
    }, f'model_with_multiplier_{str(multiplier)}.pth')

    mg = init_mg(model)
    optimizer = optim.Adam(mg.model.parameters(), lr=0.001)
'''

'\nimport torch\nfrom torchmetrics.classification import MulticlassAccuracy\nfrom torch import optim\n\ndef init_mg():\n    model = JSC_Three_Linear_Layers()\n    mg = MaseGraph(model=model)\n    mg, _ = init_metadata_analysis_pass(mg, None)\n    return mg\n\nchannel_multiplier = [1,2,3,4,5]\nmax_epoch=10\n\nbatch_size = 512\noptimizer = optim.Adam(model.parameters(), lr=0.001)\n\nfor multiplier in channel_multiplier:\n    mg = init_mg()\n    pass_config_linear["seq_blocks_2"]["config"]["channel_multiplier"] = multiplier\n    pass_config_linear["seq_blocks_4"]["config"]["channel_multiplier"] = multiplier\n    pass_config_linear["seq_blocks_6"]["config"]["channel_multiplier"] = multiplier\n\n    mg, _ = redefine_linear_transform_pass(graph=mg, pass_args={"config": pass_config_linear})\n    mg, _ = redefine_relu_pass(graph=mg, pass_args={"config": pass_config_relu})\n\n    for epoch in range(max_epoch):\n        for inputs in data_module.train_dataloader():\n            xs, ys = inputs\n

In [9]:
# 2. Grid Search for the best channel multiplier value

# search

import torch
from torchmetrics.classification import MulticlassAccuracy
import gc

def init_mg():
    model = JSC_Three_Linear_Layers()
    mg = MaseGraph(model=model)
    mg, _ = init_metadata_analysis_pass(mg, None)
    return mg

channel_multiplier = [1,2,3,4,5]
recorded_accs = []
metric = MulticlassAccuracy(num_classes=5)

for multiplier in channel_multiplier:
    mg = init_mg()
    pass_config_linear["seq_blocks_2"]["config"]["channel_multiplier"] = multiplier
    pass_config_linear["seq_blocks_4"]["config"]["channel_multiplier"] = multiplier
    pass_config_linear["seq_blocks_6"]["config"]["channel_multiplier"] = multiplier

    mg, _ = redefine_linear_transform_pass(graph=mg, pass_args={"config": pass_config_linear})
    mg, _ = redefine_relu_pass(graph=mg, pass_args={"config": pass_config_relu})

    mymodel = load_model(f"mase_output/4_2/model_with_multiplier_{multiplier}.ckpt", "pl", mg.model)

    acc_avg, loss_avg = 0, 0 
    accs, losses = [], []
    for inputs in data_module.train_dataloader():
        xs, ys = inputs
        preds = mymodel(xs)
        acc = metric(preds, ys)
        accs.append(acc)
        loss = torch.nn.functional.cross_entropy(preds, ys)
        losses.append(loss)

    acc_avg = sum(accs) / len(accs)
    loss_avg = sum(losses) / len(losses)
    recorded_accs.append({"multiplier":multiplier, "acc":acc_avg.item(),"loss":loss_avg.item()})

    del mymodel, mg, accs, losses, acc_avg, loss_avg
    gc.collect()

INFO     Loaded pytorch lightning checkpoint from mase_output/4_2/model_with_multiplier_1.ckpt


main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier': 1}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier': 1}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier': 1}}}


INFO     Loaded pytorch lightning checkpoint from mase_output/4_2/model_with_multiplier_2.ckpt


main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier': 2}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier': 2}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier': 2}}}


INFO     Loaded pytorch lightning checkpoint from mase_output/4_2/model_with_multiplier_3.ckpt


main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier': 3}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier': 3}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier': 3}}}


INFO     Loaded pytorch lightning checkpoint from mase_output/4_2/model_with_multiplier_4.ckpt


main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier': 4}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier': 4}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier': 4}}}


INFO     Loaded pytorch lightning checkpoint from mase_output/4_2/model_with_multiplier_5.ckpt


main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier': 5}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier': 5}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier': 5}}}


In [10]:
metrics = pd.DataFrame(recorded_accs)

metrics

,multiplier,acc,loss
0,1,0.185062,1.615053
1,2,0.205691,1.611572
2,3,0.118622,1.619060
3,4,0.123247,1.607118
4,5,0.234985,1.604752


In [9]:
# 3. More advanced search

import copy
import torch
import torch.optim as optim

# rewrite linear_transform
def redefine_linear_transform_pass(graph, pass_args=None):
    pass_args_copy = copy.deepcopy(pass_args)
    main_config = pass_args_copy.pop('config')
    print("main_config",main_config)
    default = main_config.pop('default', None)
    if default is None:
        raise ValueError(f"default value must be provided.")
    i = 0
    for node in graph.fx_graph.nodes:
        i += 1
        # if node name is not matched, it won't be tracked
        config = main_config.get(node.name, default)['config']
        name = config.get("name", None)
        if name is not None:
            ori_module = graph.modules[node.target]
            in_features = ori_module.in_features   
            out_features = ori_module.out_features
            bias = ori_module.bias
            if name == "output_only":
                out_features = out_features * config["channel_multiplier_output"] 
            elif name == "both":
                in_features = in_features * config["channel_multiplier_input"] 
                out_features = out_features * config["channel_multiplier_output"] 
            elif name == "input_only":
                in_features = in_features * config["channel_multiplier_input"] 
                # bias unchanged
            new_module = instantiate_linear(in_features, out_features, bias)
            parent_name, name = get_parent_name(node.target)
            setattr(graph.modules[parent_name], name, new_module)
    return graph, {}

def init_mg():
    model = JSC_Three_Linear_Layers()
    mg = MaseGraph(model=model)
    mg, _ = init_metadata_analysis_pass(mg, None)
    return mg

def design_pass_config_linear(a,b,c,d):
    config = {
        "by": "name",
        "default": {"config": {"name": None}},
        "seq_blocks_2": {
            "config": {
                "name": "output_only",
                "channel_multiplier_output": a,
            }
        },
        "seq_blocks_4": {
            "config": {
                "name": "both",
                "channel_multiplier_input": b,
                "channel_multiplier_output": c,
            }
        },
        "seq_blocks_6": {
            "config": {
                "name": "input_only",
                "channel_multiplier_input": d,
            }
        },
    }
    return config


# train

'''
multipliers = [1, 2, 3, 4, 5]

max_epoch=10
batch_size = 512
optimizer = optim.Adam(model.parameters(), lr=1e-5)

for a in multipliers:
    for c in multipliers:
        b = a
        d = c
        pass_config_linear = design_pass_config_linear(a, b, c, d)

        mg = init_mg()
        mg, _ = redefine_linear_transform_pass(mg, pass_args={"config": pass_config_linear})
        mg, _ = redefine_relu_pass(mg, pass_args={"config": pass_config_relu})

        for epoch in range(max_epoch):
            for inputs in data_module.train_dataloader():
                xs, ys = inputs
                optimizer.zero_grad()
                preds = mg.model(xs)
                loss = torch.nn.functional.cross_entropy(preds, ys)  
                loss.backward()  
                optimizer.step() 

        torch.save({
            'state_dict': mg.model.state_dict(),
            'config': [a,b,c,d],
        }, f'model_with_multiplier_{a}_{b}_{c}_{d}.ckpt')

        mg = init_mg(model)
        optimizer = optim.Adam(mg.model.parameters(), lr=1e-5)
'''

'\nmultipliers = [1, 2, 3, 4, 5]\n\nmax_epoch=10\nbatch_size = 512\noptimizer = optim.Adam(model.parameters(), lr=1e-5)\n\nfor a in multipliers:\n    for c in multipliers:\n        b = a\n        d = c\n        pass_config_linear = design_pass_config_linear(a, b, c, d)\n\n        mg = init_mg()\n        mg, _ = redefine_linear_transform_pass(mg, pass_args={"config": pass_config_linear})\n        mg, _ = redefine_relu_pass(mg, pass_args={"config": pass_config_relu})\n\n        for epoch in range(max_epoch):\n            for inputs in data_module.train_dataloader():\n                xs, ys = inputs\n                optimizer.zero_grad()\n                preds = mg.model(xs)\n                loss = torch.nn.functional.cross_entropy(preds, ys)  \n                loss.backward()  \n                optimizer.step() \n\n        torch.save({\n            \'state_dict\': mg.model.state_dict(),\n            \'config\': [a,b,c,d],\n        }, f\'model_with_multiplier_{a}_{b}_{c}_{d}.ckpt\')\n\n  

In [16]:
# train

multipliers = [1, 2, 3, 4, 5]

from torchmetrics.classification import MulticlassAccuracy
metric = MulticlassAccuracy(num_classes=5)

max_epoch=10
batch_size = 512
optimizer = optim.Adam(model.parameters(), lr=1e-5)

for a in multipliers:
    for c in multipliers:
        b = a
        d = c
        pass_config_linear = design_pass_config_linear(a, b, c, d)

        mg = init_mg()
        mg, _ = redefine_linear_transform_pass(mg, pass_args={"config": pass_config_linear})
        mg, _ = redefine_relu_pass(mg, pass_args={"config": pass_config_relu})

        for epoch in range(max_epoch):
            data_iterator = iter(data_module.train_dataloader())
            inputs = next(data_iterator)
            xs, ys = inputs
            optimizer.zero_grad()
            preds = mg.model(xs)
            loss = torch.nn.functional.cross_entropy(preds, ys)  
            loss.backward()  
            optimizer.step()


        with torch.no_grad():
            for inputs in data_module.train_dataloader():
                xs, ys = inputs
                preds = mg.model(xs)
                acc = metric(preds, ys)
                loss = torch.nn.functional.cross_entropy(preds, ys)

        print(f"acc_{a}_{b}_{c}_{d}",acc)

        mg = init_mg()
        optimizer = optim.Adam(mg.model.parameters(), lr=1e-5)

main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier_output': 1}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier_input': 1, 'channel_multiplier_output': 1}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier_input': 1}}}
acc_1_1_1_1 tensor(0.2019)
main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier_output': 1}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier_input': 1, 'channel_multiplier_output': 2}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier_input': 2}}}
acc_1_1_2_2 tensor(0.1955)
main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier_output': 1}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier_input': 1, 'channel_multiplier_output': 3}}, 

In [12]:
# 3. More advanced search

# search

import torch
from torchmetrics.classification import MulticlassAccuracy
import gc

multipliers = [1, 2, 3, 4, 5]

recorded_metrics = []
metric = MulticlassAccuracy(num_classes=5)

for a in multipliers:
    for c in multipliers:
        b = a
        d = c
        pass_config_linear = design_pass_config_linear(a, b, c, d)

        mg = init_mg()
        mg, _ = redefine_linear_transform_pass(mg, pass_args={"config": pass_config_linear})
        mg, _ = redefine_relu_pass(mg, pass_args={"config": pass_config_relu})

        mymodel = load_model(f"mase_output/4_3/model_with_multiplier_{a}_{b}_{c}_{d}.ckpt", "pl", mg.model)

        acc_avg, loss_avg = 0, 0
        accs, losses = [], []

        with torch.no_grad():
            for inputs in data_module.train_dataloader():
                xs, ys = inputs
                preds = mg.model(xs)
                acc = metric(preds, ys)
                accs.append(acc.item())
                loss = torch.nn.functional.cross_entropy(preds, ys)
                losses.append(loss.item())

        acc_avg = sum(accs) / len(accs)
        loss_avg = sum(losses) / len(losses)
        recorded_metrics.append({"block2_output":a, "block4_input":b, "block4_output":c, "block6_input":d, "acc(%)":acc_avg*100, "loss":loss_avg})

        del mymodel, mg, accs, losses, acc_avg, loss_avg
        gc.collect()


INFO     Loaded pytorch lightning checkpoint from mase_output/4_3/model_with_multiplier_1_1_1_1.ckpt


main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier_output': 1}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier_input': 1, 'channel_multiplier_output': 1}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier_input': 1}}}


INFO     Loaded pytorch lightning checkpoint from mase_output/4_3/model_with_multiplier_1_1_2_2.ckpt


main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier_output': 1}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier_input': 1, 'channel_multiplier_output': 2}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier_input': 2}}}


INFO     Loaded pytorch lightning checkpoint from mase_output/4_3/model_with_multiplier_1_1_3_3.ckpt


...

main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier_output': 5}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier_input': 5, 'channel_multiplier_output': 3}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier_input': 3}}}


INFO     Loaded pytorch lightning checkpoint from mase_output/4_3/model_with_multiplier_5_5_4_4.ckpt


main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier_output': 5}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier_input': 5, 'channel_multiplier_output': 4}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier_input': 4}}}


INFO     Loaded pytorch lightning checkpoint from mase_output/4_3/model_with_multiplier_5_5_5_5.ckpt


main_config {'by': 'name', 'default': {'config': {'name': None}}, 'seq_blocks_2': {'config': {'name': 'output_only', 'channel_multiplier_output': 5}}, 'seq_blocks_4': {'config': {'name': 'both', 'channel_multiplier_input': 5, 'channel_multiplier_output': 5}}, 'seq_blocks_6': {'config': {'name': 'input_only', 'channel_multiplier_input': 5}}}


In [13]:
# recorded_metrics

metrics = pd.DataFrame(recorded_metrics)

metrics

,block2_output,block4_input,block4_output,block6_input,acc(%),loss
0,1,1,1,1,20.629313,1.607766
1,1,1,2,2,22.239536,1.603152
2,1,1,3,3,20.041148,1.603113
3,1,1,4,4,32.193983,1.597404
4,1,1,5,5,16.924062,1.612282
5,2,2,1,1,19.999228,1.617489
6,2,2,2,2,20.454291,1.614191
7,2,2,3,3,21.310789,1.614070
8,2,2,4,4,19.583254,1.604264
9,2,2,5,5,22.100852,1.601225


In [1]:
%cd machop

/mnt/d/imperial/second_term/adls/rs1923/mase_real/machop


In [2]:
# 4. Integrate the search to the chop flow, so we can run it from the command line.

!./ch search --config configs/examples/jsc_network_search.toml --load /mnt/d/imperial/second_term/adls/rs1923/mase_real/mase_output/jsc-three-linear-layers_classification_jsc_2024-02-10/software/training_ckpts/best.ckpt
# comparison on the performance: using xxx & not using xxx

Total number of JSC_1923 parameters: 3285
Total number of JSC_Tiny parameters: 117
Seed set to 0
+-------------------------+--------------------------+--------------------------+--------------------------+--------------------------+
| Name                    |         Default          |       Config. File       |     Manual Override      |        Effective         |
+-------------------------+--------------------------+--------------------------+--------------------------+--------------------------+
| task                    |      classification      |           cls            |                          |           cls            |
| load_name               |           None           | /mnt/d/imperial/second_t | /mnt/d/imperial/second_t | /mnt/d/imperial/second_t |
|                         |                          | erm/adls/rs1923/mase_rea | erm/adls/rs1923/mase_rea | erm/adls/rs1923/mase_rea |
|                         |                          | l/mase_output/jsc-three- | l/mas

In [1]:
%cd machop/

/mnt/d/imperial/second_term/adls/rs1923/mase_real/machop


In [2]:
# optional task
# cifar10 + vgg7 + tpe

!./ch search --config configs/examples/cifar10_vgg.toml --load ../mase_output/vgg7_classification_cifar10_2024-02-01/software/training_ckpts/best.ckpt

Total number of JSC_1923 parameters: 3285
Total number of JSC_Tiny parameters: 117
Seed set to 0
+-------------------------+--------------------------+--------------------------+--------------------------+--------------------------+
| Name                    |         Default          |       Config. File       |     Manual Override      |        Effective         |
+-------------------------+--------------------------+--------------------------+--------------------------+--------------------------+
| task                    |      classification      |           cls            |                          |           cls            |
| load_name               |           None           | ../mase_output/vgg7_clas | /mnt/d/imperial/second_t | /mnt/d/imperial/second_t |
|                         |                          | sification_cifar10_2024- | erm/adls/rs1923/mase_rea | erm/adls/rs1923/mase_rea |
|                         |                          | 02-01/software/training_ | l/mas